In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import matplotlib.gridspec as gridspec

In [ ]:
folder = '/Users/vytis/Desktop/Muon Lab Data'

In [ ]:
dt = []

for i in range(3838):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue
    message = f'{filename} is bad'
    data = pd.read_csv(
        filename, skiprows=4,
    )
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        print(message)
        continue
    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()
    if len(between) != 1:
        print(message)
        continue
    Time = data['Time'].values
    a = Time[argwhere[0]].item()
    b = Time[argwhere[between + 1]].item()
    dt.append(b - a)

In [ ]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.25, hspace=0.25)
ax = fig.add_subplot(gs[0, 0])

ax.plot(data['Time'], data['Ampl'] - data['Ampl'][:1000].mean())
ax.set_xlabel('time [ns]')
ax.set_ylabel('voltage [unit]')

plt.show()

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np

# Original folder
folder = '/Users/vytis/Desktop/Muon Lab Data'

# New folder for non-bad files
new_folder = '/Users/vytis/Desktop/NonBadFiles'

# Create the new folder if it doesn't exist
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

dt = []

for i in range(8285):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue
    message = f'{filename} is bad'
    data = pd.read_csv(filename, skiprows=4)
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        print(message)
        continue
    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()
    if len(between) != 1:
        print(message)
        continue
    Time = data['Time'].values
    a = Time[argwhere[0]].item()
    b = Time[argwhere[between + 1]].item()
    dt.append(b - a)

    # Do not copy bad files to the new folder
    # Copy only non-bad files
    shutil.copy(filename, os.path.join(new_folder, f'C{1800001 + i}.csv'))

In [ ]:
import os
import pandas as pd
import numpy as np

# Original folder
folder = '/Users/vytis/Desktop/Good Muon Data'

# List to store time differences
all_time_differences = []

for i in range(7815):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue

    data = pd.read_csv(filename, skiprows=4)
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        continue

    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()

    # Ensure there is exactly one peak separation
    if len(between) != 1:
        continue

    Time = data['Time'].values
    a = Time[argwhere[0]].item()
    b = Time[argwhere[between + 1]].item()

    # Calculate time difference and store it
    time_difference = b - a
    all_time_differences.append(time_difference)

# Save time differences to a file on the desktop
output_file_path = '/Users/vytis/Desktop/time_differences_good_data.txt'
with open(output_file_path, 'w') as file:
    for time_difference in all_time_differences:
        file.write(f'{time_difference}\n')

print(f'Time differences saved to {output_file_path}')

In [ ]:
# File containing time differences
input_file_path = '/Users/vytis/Desktop/time_differences_good_data.txt'

# Check if the file exists
if not os.path.exists(input_file_path):
    print(f'The file {input_file_path} does not exist.')
else:
    # Read time differences from the file
    with open(input_file_path, 'r') as file:
        time_differences = [float(line.strip()) for line in file]
        
        # Print the total number of entries
    total_entries = len(time_differences)
    print(f'Total number of entries: {total_entries}')

    # Calculate and print the mean
    if time_differences:
        average_time_difference = np.mean(time_differences)
        max_time_difference = max(time_differences)
        max_index = np.argmax(time_differences)
        print(f'Average time difference: {average_time_difference} seconds')
        print(max_time_difference)
        print(max_index)
    else:
        print('No time differences found in the file.')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# File containing decay times
file_path = '/Users/vytis/Desktop/Time Differences/2023-10-14_time_differences.txt'

# Read decay times from the file
with open(file_path, 'r') as file:
    decay_times = [float(line.strip()) for line in file]

# Define the exponential decay function
def exponential_decay(t, N0, t0):
    return N0 * np.exp(-t / t0)

# Bin the decay times for the histogram
bins = np.linspace(0, max(decay_times), 50)

# for the first 1000 lines make a histogram
for i in range(len(decay_times) // 1000):
    # Plot the histogram
    plt.hist(
        decay_times[i * 1000:(i + 1) * 1000], bins=bins,
        histtype='step',
        alpha=0.5, label='Histogram of Decay Times',
    )
plt.title('Histogram of Muon Decay Times')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
plt.legend()

# Fit the data to the exponential decay model
popt, pcov = curve_fit(exponential_decay, bins[:-1], np.histogram(decay_times, bins=bins)[0])

# Plot the fitted exponential decay curve
fit_times = np.linspace(0, max(decay_times), 100)
#plt.plot(fit_times, exponential_decay(fit_times, *popt), 'r-', label=f'Exponential Fit (t0 = {popt[1]:.2f} seconds)')

# Show the plot
plt.legend()
plt.show()

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Specify the path to your CSV file
folder = '/Users/vytis/Desktop/Good Muon Data'
filename = os.path.join(folder, 'C1800032.csv')

# Read the CSV file
data = pd.read_csv(filename, skiprows=4)

baseline = data['Ampl'][:1000].mean()
assert baseline <= -0.5

argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
between = np.argwhere(np.diff(argwhere) > 2).flatten()

# Ensure there is exactly one peak separation
assert len(between) == 1

Time = data['Time'].values
a = Time[argwhere[0]].item()
b = Time[argwhere[between + 1]].item()

# Calculate time difference
time_difference = b - a

# Plot the data
fig = plt.figure(figsize=(8, 6))
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.25, hspace=0.25)
ax = fig.add_subplot(gs[0, 0])

ax.plot(data['Time'], data['Ampl'] - data['Ampl'][:1000].mean())
ax.set_xlabel('time [s]')
ax.set_ylabel('voltage [unit]')
# ax.set_title(f'Plot for {filename}')
ax.axvline(-1e-6, color='k')
ax.set_title(f'{time_difference}')

plt.show()

In [ ]:
import os
import pandas as pd
import numpy as np

# Original folder
folder = '/Users/vytis/Desktop/Good Muon Data'
#folder = '/Users/vytis/Desktop/AC'

time_differences = []

# Iterate over files
for i in range(15):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue

    data = pd.read_csv(filename, skiprows=4)
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        continue

    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()

    # Ensure there is exactly one peak separation
    if len(between) != 1:
        continue

    Time = data['Time'].values
    a = Time[argwhere[0]].item()
    b = Time[argwhere[between + 1]].item()

    # Calculate time difference
    time_difference = b - a
    time_differences.append(time_difference)

    # Print the filename if the time difference is greater than 8e-06
#     if time_difference > 8e-06:
#         print(f'File with time difference > 8e-06: {filename}')

print('Processing complete')

In [ ]:
plt.hist(
    np.random.exponential(1 / 2.2, size=100000),
    alpha=0.5, label='Histogram of Decay Times',
)

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Original folder
folder = '/Users/vytis/Desktop/Good Muon Data'

# Dictionary to store time differences by day
time_differences_by_day = {}

for i in range(7815):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue

    data = pd.read_csv(filename, skiprows=4)
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        continue

    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()

    # Ensure there is exactly one peak separation
    if len(between) != 1:
        continue

    data1 = pd.read_csv(filename)
    # Extract date from the data file
    date_str = str(data1.iloc[2, 0])  # Convert to string
    date = datetime.strptime(date_str, '%d-%b-%Y %H:%M:%S').date()
    
    print(date_str)

    Time = data['Time'].values
    a = Time[argwhere[0]].item()
    b = Time[argwhere[between + 1]].item()

    # Calculate time difference and store it in the dictionary
    time_difference = b - a
    if date not in time_differences_by_day:
        time_differences_by_day[date] = []
    time_differences_by_day[date].append(time_difference)

# Save time differences to files organized by day
output_folder = '/Users/vytis/Desktop/Time Differences'
os.makedirs(output_folder, exist_ok=True)

for date, time_differences in time_differences_by_day.items():
    output_file_path = os.path.join(output_folder, f'{date}_time_differences.txt')
    with open(output_file_path, 'w') as file:
        for time_difference in time_differences:
            file.write(f'{time_difference}\n')

    print(f'Time differences for {date} saved to {output_file_path}')

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Original folder
folder = '/Users/vytis/Desktop/Good Muon Data'

# Dictionary to store time differences by day
time_differences_by_day = {}

for i in range(100):
    filename = os.path.join(folder, f'C{1800001 + i}.csv')
    if not os.path.exists(filename):
        continue

    data = pd.read_csv(filename, skiprows=4)
    Time = data['Time'].values
    print(Time[0])
    baseline = data['Ampl'][:1000].mean()
    if baseline > -0.5:
        continue

    argwhere = np.argwhere(data['Ampl'].values - baseline > 0.2).flatten()
    between = np.argwhere(np.diff(argwhere) > 2).flatten()

    # Ensure there is exactly one peak separation
    if len(between) != 1:
        continue

In [ ]:
# File containing decay times
file_path = '/Users/vytis/Desktop/Time Differences/2023-10-14_time_differences.txt'

# Read decay times from the file
with open(file_path, 'r') as file:
    decay_times = np.array([float(line.strip()) for line in file])

# Calculate average time, quartiles, and median
average_time = np.mean(decay_times)
quartiles = np.percentile(decay_times, [25, 50, 75])
median = np.median(decay_times)

# Define the exponential decay function
def exponential_decay(t, N0, t0):
    return N0 * np.exp(-t / t0)

# Bin the decay times for the histogram
bins = np.linspace(0, max(decay_times), 50)

# Plot the histogram
plt.hist(decay_times, bins=bins, histtype='step', alpha=0.5)
plt.axvline(average_time, color='r', linestyle='--', label=f'Average Time: {average_time:.2e} seconds')
plt.axvline(quartiles[0], color='g', linestyle='--', label=f'Median: {median:.2e} seconds')
plt.axvline(median, color='b', linestyle='--', label=f'First Quartile: {quartiles[0]:.2e} seconds')
plt.axvline(quartiles[2], color='orange', linestyle='--', label=f'Third Quartile: {quartiles[2]:.2e} seconds')

# Shade the quartile regions
plt.fill_betweenx(y=[0, plt.ylim()[1]], x1=0, x2=quartiles[0], color='green', alpha=0.2)
plt.fill_betweenx(y=[0, plt.ylim()[1]], x1=quartiles[0], x2=median, color='blue', alpha=0.2)
plt.fill_betweenx(y=[0, plt.ylim()[1]], x1=median, x2=quartiles[2], color='orange', alpha=0.2)
plt.fill_betweenx(y=[0, plt.ylim()[1]], x1=quartiles[2], x2=max(decay_times), color='red', alpha=0.2)

plt.title('Histogram of Muon Decay Times')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
plt.legend()

# Fit the data to the exponential decay model with initial parameter values
hist, bin_edges = np.histogram(decay_times, bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
initial_guess = [max(hist), 2.97e-06]  # Initial guess for N0 and t0
popt, pcov = curve_fit(exponential_decay, bin_centers, hist, p0=initial_guess)

# Plot the fitted exponential decay curve
fit_times = np.linspace(0, max(decay_times), 100)
plt.plot(fit_times, exponential_decay(fit_times, *popt), 'r-', label=f'Exponential Fit (t0 = {popt[1]:.2e} seconds)')

# Show the plot
plt.legend()
plt.show()

# Print calculated values
print(max(hist))
print(np.mean(decay_times))
print(f'Average Time: {average_time:.2e} seconds')
print(f'Median: {median:.2e} seconds')
print(f'First Quartile: {quartiles[0]:.2e} seconds')
print(f'Third Quartile: {quartiles[2]:.2e} seconds')

In [ ]:
# File containing decay times
file_path = '/Users/vytis/Desktop/Time Differences/Background Corrections.txt'

# Read decay times from the file
with open(file_path, 'r') as file:
    decay_times = [float(line.strip()) for line in file]

# Define the exponential decay function
def exponential_decay(t, N0, t0):
    return N0 * np.exp(-t / t0)

# Bin the decay times for the histogram
bins = np.linspace(0, max(decay_times), 50)

# Plot the histogram
plt.hist(decay_times, bins=bins, histtype='step', alpha=0.5)
plt.title('Histogram of Background Correction')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
#plt.legend()

# Fit the data to the exponential decay model with initial parameter values
hist, bin_edges = np.histogram(decay_times, bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
initial_guess = [max(hist), np.mean(decay_times)]  # Initial guess for N0 and t0
popt, pcov = curve_fit(exponential_decay, bin_centers, hist, p0=initial_guess)

# Plot the fitted exponential decay curve
fit_times = np.linspace(0, max(decay_times), 100)
#plt.plot(fit_times, exponential_decay(fit_times, *popt), 'r-', label=f'Exponential Fit (t0 = {popt[1]:.2e} seconds)')

# Show the plot
#plt.legend()
plt.show()

In [ ]:
# File containing decay times
file_path = '/Users/vytis/Desktop/Time Differences/2023-10-14_time_differences.txt'

# Read decay times from the file
with open(file_path, 'r') as file:
    decay_times = np.array([float(line.strip()) for line in file])

# Bin the decay times for the histogram
bins = np.linspace(0, max(decay_times), 50)

# Plot the histogram
hist, bin_edges, _ = plt.hist(decay_times, bins=bins, histtype='step', alpha=0.5)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Define the Poisson log-likelihood function for exponential decay
def poisson_log_likelihood(params, t, counts):
    N0, t0, c = params
    expected_counts = N0 * np.exp(-t / t0) + c
    log_likelihood = -np.sum(counts * np.log(expected_counts) - expected_counts)
    return log_likelihood

# Provide initial guesses
initial_guess = [max(hist), np.mean(decay_times), min(hist)]

# Minimize the negative log-likelihood
result = minimize(poisson_log_likelihood, initial_guess, args=(bin_centers, hist), method='L-BFGS-B')

# Extract the optimized parameters
popt = result.x

# Plot the histogram
plt.hist(decay_times, bins=bins, histtype='step', alpha=0.5)

# Plot the fitted exponential decay curve
fit_times = np.linspace(0, max(decay_times), 100)
plt.plot(fit_times, popt[0] * np.exp(-fit_times / popt[1]) + popt[2], 'r-', label=f'Exponential Fit (t0 = {popt[1]:.2e} seconds)')

# Show the plot
plt.title('Histogram of Muon Decay Times')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Print calculated values
print(f'Exponential Fit Parameters: N0 = {popt[0]:.2e}, t0 = {popt[1]:.2e}, c = {popt[2]:.2e}')
print(np.mean(decay_times))

In [ ]:
# File containing decay times
file_path = '/Users/vytis/Desktop/Time Differences/2023-10-14_time_differences.txt'

# Read decay times from the file
with open(file_path, 'r') as file:
    decay_times = np.array([float(line.strip()) for line in file])

# Singles rate correction
singles_rate = 131 / 600  # Adjust this based on your actual singles rate
delta_t = 2.965856629402971e-06  # mean decay time of muon
sing_rate_adj = (singles_rate**2) * delta_t  # total of all counts

# Corrected decay times
corrected_decay_times = decay_times - sing_rate_adj

# Bin the decay times for the histogram
bins = np.linspace(0, max(corrected_decay_times), 50)

# Plot the histogram
hist, bin_edges, _ = plt.hist(corrected_decay_times, bins=bins, histtype='step', alpha=0.5)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Define the Poisson log-likelihood function for exponential decay
def poisson_log_likelihood(params, t, counts):
    N0, t0, c = params
    expected_counts = N0 * np.exp(-t / t0) + c
    log_likelihood = -np.sum(counts * np.log(expected_counts) - expected_counts)
    return log_likelihood

# Provide initial guesses
initial_guess = [max(hist), np.mean(corrected_decay_times), min(hist)]

# Minimize the negative log-likelihood
result = minimize(poisson_log_likelihood, initial_guess, args=(bin_centers, hist), method='L-BFGS-B')

# Extract the optimized parameters
popt = result.x

# Plot the histogram
plt.hist(corrected_decay_times, bins=bins, histtype='step', alpha=0.5)

# Plot the fitted exponential decay curve
fit_times = np.linspace(0, max(corrected_decay_times), 100)
plt.plot(fit_times, popt[0] * np.exp(-fit_times / popt[1]) + popt[2], 'r-', label=f'Exponential Fit (t0 = {popt[1]:.2e} seconds)')

# Show the plot
plt.title('Histogram of Muon Decay Times (Corrected for Singles Rate)')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Calculate and print the mean time divided by the square root of the total number of decay times
mean_decay_time_corrected = np.mean(corrected_decay_times)
sqrt_total_decays = np.sqrt(len(corrected_decay_times))
mean_time_divided_by_sqrt_total = mean_decay_time_corrected / sqrt_total_decays
print(f'Exponential Fit Parameters: N0 = {popt[0]:.2e}, t0 = {popt[1]:.2e}, c = {popt[2]:.2e}')
print(f'Mean Decay Time (Corrected): {mean_decay_time_corrected:.2e} seconds')
print(f'Mean Time / sqrt(Total Decay Times): {mean_time_divided_by_sqrt_total:.2e} seconds')

In [ ]:
# Manually input the variables for the fit
N0 = 104.0  # Initial number of events
t0 = 2.58e-6  # Decay time constant
background_rate = -2.51e-15  # Background rate

# File containing decay times
file_path_data = '/Users/vytis/Desktop/Time Differences/2023-10-14_time_differences.txt'
file_path_background = '/Users/vytis/Desktop/Time Differences/Background Corrections.txt'

# Read decay times from the data file
with open(file_path_data, 'r') as file:
    decay_times_data = np.array([float(line.strip()) for line in file])

# Read decay times from the background file
with open(file_path_background, 'r') as file:
    decay_times_background = np.array([float(line.strip()) for line in file])

# Calculate mean time interval and rate for the background
mean_interval_background = np.mean(np.diff(decay_times_background))
rate_background = 1 / mean_interval_background

# Correct the data for background
corrected_decay_times = decay_times_data - mean_interval_background

# Define the exponential decay function with background correction
def exponential_decay_with_background(t, N0, t0, background_rate):
    return N0 * np.exp(-t / t0) + background_rate

# Bin the corrected decay times for the histogram
bins = np.linspace(0, max(corrected_decay_times), 50)

# Plot the histogram
hist, bin_edges, _ = plt.hist(corrected_decay_times, bins=bins, histtype='step', alpha=0.5)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram
plt.hist(corrected_decay_times, bins=bins, histtype='step', alpha=0.5)

# Plot the manually input exponential decay curve with background correction
fit_times = np.linspace(0, max(corrected_decay_times), 100)
plt.plot(fit_times, exponential_decay_with_background(fit_times, N0, t0, background_rate), 'r-', label=f'Exponential Fit (t0 = {t0:.2e} seconds)')

# Show the plot
plt.title('Histogram of Muon Decay Times with Background Correction')
plt.xlabel('Decay Time (seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Calculate and print the mean time divided by the square root of the total number of decay times
mean_decay_time_corrected = np.mean(corrected_decay_times)
sqrt_total_decays = np.sqrt(len(corrected_decay_times))
mean_time_divided_by_sqrt_total = mean_decay_time_corrected / sqrt_total_decays
print(f'Manually Input Parameters: N0 = {N0:.2e}, t0 = {t0:.2e}, Background Rate = {background_rate:.2e}')
print(f'Mean Decay Time (Corrected): {mean_decay_time_corrected:.2e} seconds')
print(f'Mean Time / sqrt(Total Decay Times): {mean_time_divided_by_sqrt_total:.2e} seconds')